# NYC Taxi Trips 
https://www.kaggle.com/c/nyc-taxi-trip-duration/data


### Imports

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
import fastai
from fastai.structured import *
import pandas as pd
import numpy as np
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

### Load Data

#### Training Set

In [63]:
df_raw = pd.read_csv('./data/nyc-taxi/train.csv',parse_dates=['pickup_datetime','dropoff_datetime'])

In [64]:
df_raw.dtypes

id                            object
vendor_id                      int64
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag            object
trip_duration                  int64
dtype: object

In [65]:
len(df_raw)

1458644

In [66]:
len(df_raw.columns)

11

In [67]:
df_summ = DataFrameSummary(df_raw)
display_all(df_summ.summary())

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
count,NaN,1.45864e+06,NaN,NaN,1.45864e+06,1.45864e+06,1.45864e+06,1.45864e+06,1.45864e+06,NaN,1.45864e+06
mean,NaN,1.53495,NaN,NaN,1.66453,-73.9735,40.7509,-73.9734,40.7518,NaN,959.492
std,NaN,0.498777,NaN,NaN,1.31424,0.0709019,0.0328812,0.0706433,0.0358906,NaN,5237.43
min,NaN,1,NaN,NaN,0,-121.933,34.3597,-121.933,32.1811,NaN,1
25%,NaN,1,NaN,NaN,1,-73.9919,40.7373,-73.9913,40.7359,NaN,397
50%,NaN,2,NaN,NaN,1,-73.9817,40.7541,-73.9798,40.7545,NaN,662
75%,NaN,2,NaN,NaN,2,-73.9673,40.7684,-73.963,40.7698,NaN,1075
max,NaN,2,NaN,NaN,9,-61.3355,51.8811,-61.3355,43.921,NaN,3.52628e+06
counts,1458644,1458644,1458644,1458644,1458644,1458644,1458644,1458644,1458644,1458644,1458644
uniques,1458644,2,1380222,1380377,10,23047,45245,33821,62519,2,7417


In [ ]:
types = {'id': 'unit32', 'vendor_id': '', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'}

In [19]:
df_raw[['pickup_datetime','dropoff_datetime']]

,pickup_datetime,dropoff_datetime
0,2016-03-14 17:24:55,2016-03-14 17:32:30
1,2016-06-12 00:43:35,2016-06-12 00:54:38
2,2016-01-19 11:35:24,2016-01-19 12:10:48
3,2016-04-06 19:32:31,2016-04-06 19:39:40
4,2016-03-26 13:30:55,2016-03-26 13:38:10
...,...,...
1458639,2016-04-08 13:31:04,2016-04-08 13:44:02
1458640,2016-01-10 07:35:15,2016-01-10 07:46:10
1458641,2016-04-22 06:57:41,2016-04-22 07:10:25
1458642,2016-01-05 15:56:26,2016-01-05 16:02:39


In [21]:
df_raw.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


#### Test Set

In [68]:
df_test = pd.read_csv('./data/nyc-taxi/test.csv')

In [69]:
df_test_summ = DataFrameSummary(df_test)
display_all(df_test_summ .summary())

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
count,NaN,625134,NaN,625134,625134,625134,625134,625134,NaN
mean,NaN,1.53488,NaN,1.66177,-73.9736,40.7509,-73.9735,40.7518,NaN
std,NaN,0.498782,NaN,1.31129,0.073389,0.0298478,0.0725649,0.0358239,NaN
min,NaN,1,NaN,0,-121.933,37.3896,-121.933,36.6013,NaN
25%,NaN,1,NaN,1,-73.9919,40.7374,-73.9913,40.736,NaN
50%,NaN,2,NaN,1,-73.9817,40.7541,-73.9798,40.7545,NaN
75%,NaN,2,NaN,2,-73.9674,40.7684,-73.963,40.7699,NaN
max,NaN,2,NaN,9,-69.2489,42.8149,-67.4968,48.8576,NaN
counts,625134,625134,625134,625134,625134,625134,625134,625134,625134
uniques,625134,2,610581,8,18926,38408,26476,51439,2


In [70]:
df_test.columns # missing dropoff time and trip length

Index(['id', 'vendor_id', 'pickup_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'store_and_fwd_flag'],
      dtype='object')

### Cleaning and Transforms

#### Remove cols from training set to match test set

In [72]:
df_raw = df_raw.drop(columns=['dropoff_datetime'])

In [73]:
df_raw.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration'],
      dtype='object')

#### Add Datepart

In [74]:
%time add_datepart(df_raw,'pickup_datetime')
%time add_datepart(df_test,'pickup_datetime')

CPU times: user 1.52 s, sys: 190 ms, total: 1.71 s
Wall time: 1.87 s
CPU times: user 716 ms, sys: 114 ms, total: 831 ms
Wall time: 853 ms


In [75]:
display_all(df_raw)

,id,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_datetimeYear,pickup_datetimeMonth,pickup_datetimeWeek,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,pickup_datetimeElapsed
0,id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016,3,11,14,0,74,False,False,False,False,False,False,1457976295
1,id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,N,663,2016,6,23,12,6,164,False,False,False,False,False,False,1465692215
2,id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,2016,1,3,19,1,19,False,False,False,False,False,False,1453203324
3,id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,N,429,2016,4,14,6,2,97,False,False,False,False,False,False,1459971151
4,id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,N,435,2016,3,12,26,5,86,False,False,False,False,False,False,1458999055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,id2376096,2,4,-73.982201,40.745522,-73.994911,40.740170,N,778,2016,4,14,8,4,99,False,False,False,False,False,False,1460122264
1458640,id1049543,1,1,-74.000946,40.747379,-73.970184,40.796547,N,655,2016,1,1,10,6,10,False,False,False,False,False,False,1452411315
1458641,id2304944,2,1,-73.959129,40.768799,-74.004433,40.707371,N,764,2016,4,16,22,4,113,False,False,False,False,False,False,1461308261
1458642,id2714485,1,1,-73.982079,40.749062,-73.974632,40.757107,N,373,2016,1,1,5,1,5,False,False,False,False,False,False,1452009386


#### Modify dependent variable to match evaluation metric

In [76]:
df_raw.trip_duration

0           455
1           663
2          2124
3           429
4           435
           ... 
1458639     778
1458640     655
1458641     764
1458642     373
1458643     198
Name: trip_duration, Length: 1458644, dtype: int64

In [77]:
df_raw.trip_duration = np.log(df_raw.trip_duration)

In [78]:
df_raw.trip_duration

0          6.120297
1          6.496775
2          7.661056
3          6.061457
4          6.075346
             ...   
1458639    6.656727
1458640    6.484635
1458641    6.638568
1458642    5.921578
1458643    5.288267
Name: trip_duration, Length: 1458644, dtype: float64

#### split raw df into train and val sets, val set contains most recent values


In [81]:
def split_vals(a,n):
    return a[:n].copy(),a[n:].copy()

In [82]:
n_valid = len(df_test)
n_train = len(df_raw)-n_valid
training_set, val_set = split_vals(df_raw,n_train)
training_set.shape, val_set.shape

((833510, 22), (625134, 22))

#### Separate response variables in df_raw and convert everything to numeric

In [83]:
%time
X_train, y_train, _ = proc_df(training_set,y_fld='trip_duration')
X_valid, y_valid, _ = proc_df(val_set,y_fld='trip_duration')
X_test, _, _ = proc_df(df_test)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


In [84]:
X_train

,id,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_datetimeYear,pickup_datetimeMonth,...,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,pickup_datetimeElapsed
0,599533,2,1,-73.982155,40.767937,-73.964630,40.765602,1,2016,3,...,14,0,74,False,False,False,False,False,False,1457976295
1,495717,1,1,-73.980415,40.738564,-73.999481,40.731152,1,2016,6,...,12,6,164,False,False,False,False,False,False,1465692215
2,803903,2,1,-73.979027,40.763939,-74.005333,40.710087,1,2016,1,...,19,1,19,False,False,False,False,False,False,1453203324
3,730611,2,1,-74.010040,40.719971,-74.012268,40.706718,1,2016,4,...,6,2,97,False,False,False,False,False,False,1459971151
4,454722,2,1,-73.973053,40.793209,-73.972923,40.782520,1,2016,3,...,26,5,86,False,False,False,False,False,False,1458999055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833505,663114,1,1,-73.972450,40.764912,-73.991188,40.739456,1,2016,2,...,23,1,54,False,False,False,False,False,False,1456226477
833506,255389,1,1,-73.959084,40.763596,-74.008049,40.738941,1,2016,4,...,24,6,115,False,False,False,False,False,False,1461468915
833507,57563,2,1,-73.895889,40.740517,-73.923767,40.743988,1,2016,2,...,29,0,60,True,False,False,False,False,False,1456729381
833508,135462,2,1,-73.995674,40.764587,-73.994263,40.759308,1,2016,1,...,30,5,30,False,False,False,False,False,False,1454185326


In [85]:
X_test

,id,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_datetimeYear,pickup_datetimeMonth,...,pickup_datetimeDay,pickup_datetimeDayofweek,pickup_datetimeDayofyear,pickup_datetimeIs_month_end,pickup_datetimeIs_month_start,pickup_datetimeIs_quarter_end,pickup_datetimeIs_quarter_start,pickup_datetimeIs_year_end,pickup_datetimeIs_year_start,pickup_datetimeElapsed
0,469365,1,1,-73.988129,40.732029,-73.990173,40.756680,1,2016,6,...,30,3,182,True,False,True,False,False,False,1467331198
1,547622,1,1,-73.964203,40.679993,-73.959808,40.655403,1,2016,6,...,30,3,182,True,False,True,False,False,False,1467331193
2,190108,1,1,-73.997437,40.737583,-73.986160,40.729523,1,2016,6,...,30,3,182,True,False,True,False,False,False,1467331187
3,335484,2,1,-73.956070,40.771900,-73.986427,40.730469,1,2016,6,...,30,3,182,True,False,True,False,False,False,1467331181
4,249510,1,1,-73.970215,40.761475,-73.961510,40.755890,1,2016,6,...,30,3,182,True,False,True,False,False,False,1467331173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625129,470065,1,1,-74.003464,40.725105,-74.001251,40.733643,1,2016,1,...,1,4,1,False,True,False,True,False,True,1451606572
625130,578498,1,1,-74.006363,40.743782,-73.953407,40.782467,1,2016,1,...,1,4,1,False,True,False,True,False,True,1451606512
625131,401489,1,2,-73.972267,40.759865,-73.876602,40.748665,1,2016,1,...,1,4,1,False,True,False,True,False,True,1451606484
625132,216305,1,1,-73.976501,40.733562,-73.854263,40.891788,1,2016,1,...,1,4,1,False,True,False,True,False,True,1451606428


## Random Forest

In [87]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m,'oob_score_'): res.append(m.oob_score_)
    print(res)

In [88]:
m = RandomForestRegressor(n_estimators=100,n_jobs=-1,oob_score=True)
%time m.fit(X_train,y_train)
print_score(m)

CPU times: user 45min 50s, sys: 27 s, total: 46min 17s
Wall time: 12min 49s
[0.16945511982139552, 0.4521457147008209, 0.9550366876012534, 0.6807560110118979, 0.6733860996141081]


**overfitting, 94% on training set yet 66% on validation set**

### Submit Test set

In [89]:
preds = m.predict(X_test)
preds

array([7.10932, 6.31243, 6.30431, ..., 7.22141, 7.76516, 7.10233])

**convert back from log by applying exponent**

In [99]:
preds_exp = np.exp(preds).astype('int')

In [100]:
preds_exp

array([1223,  551,  546, ..., 1368, 2357, 1214])

In [101]:
len(preds)

625134

In [102]:
df_sub = pd.DataFrame({
    'id': df_test['id'].values,
    'trip_duration': preds_exp}).set_index('id')

In [103]:
df_sub.to_csv('subs/nyc001.csv')

In [104]:
df_test['id'].values

array(['id3004672', 'id3505355', 'id1217141', ..., 'id2568735', 'id1384355', 'id0621643'], dtype=object)

**score of 0.45496 in Kaggle around 50%, not great, but will score improve with feature engineering?**

## GBM

## XGBoost